In [ ]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)

In [149]:
df=pd.read_excel('◎位置情報と対応(水道水) 基準満たしているか否か.xlsx')
df=df.dropna(axis=0)

In [148]:
df=pd.read_csv('DATA.csv')
df=df.dropna(axis=0)

In [100]:
df.columns

Index(['NAME', 'Rowid', 'XY水道水位置情報', 'X', 'Y', 'up_down', 'curvature',
       'inclination', 'tilt_direction', 'altitude', 'disto_river',
       'disto_stations', 'disto_mainroad', 'disto_syorizyo', 'supply_hours',
       'no_water_days', 'total_population ', 'population_served',
       'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe',
       'served/pipes', '(popu-served)/pipes', 'oldest_pipe_age', 'ST', 'RSF',
       'FL', 'PF', 'RF', 'EColi.'],
      dtype='object')

In [150]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes', 
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF']]

In [133]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes', 
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF', 'ratio', 'source_ecoli']]

In [151]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [152]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

In [153]:
def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1

In [154]:
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

In [ ]:
X_train_std=X_train_std.drop('ratio', axis=1)
X_train_std=pd.concat(X_train_std, X_train['ratio'])
X_train_std=pd.concat([X_train_std, X_train['ratio']], axis=1)

In [155]:
X_train_std.head()

,up_down,curvature,inclination,tilt_direction,altitude,disto_river,disto_stations,disto_mainroad,disto_syorizyo,supply_hours,no_water_days,total_population,population_served,popu-served,number_taps,pipelength,pipelength_per_pipe,served/pipes,(popu-served)/pipes,oldest_pipe_age,ST,RSF,FL,PF,RF
218,-0.686850,-0.382187,0.048990,-0.410915,-1.159545,-0.327503,-1.265976,-0.135508,-0.070461,0.939600,-1.478259,-1.032852,-1.008378,-1.129822,-1.022905,-1.036154,-0.031214,-1.256239,-0.508817,-0.784953,1,1,0,0,0
100,1.831442,-1.612959,0.077027,1.317255,0.879267,0.004160,0.949064,-0.448003,-0.178807,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1
147,-0.686850,0.110122,0.083950,0.323822,0.939576,0.527216,1.045501,-0.147035,-0.094643,-0.732687,0.484507,1.008671,1.009306,0.995391,1.008055,1.006044,-0.190579,0.969445,-0.352555,1.003462,1,1,1,1,1
190,-0.327094,0.725508,0.080769,0.610908,-1.188024,-0.724693,-0.980053,-0.163948,-0.394109,0.380087,-0.877575,-1.024947,-1.014043,-1.062279,-1.033523,-1.070890,-0.023260,-1.165195,0.956916,-1.091538,0,0,0,0,0
192,-0.226362,-0.012955,0.082693,-1.455383,-1.146142,-0.883740,-1.183121,1.309977,-0.016418,-0.515133,0.040044,-0.925869,-0.999170,-0.593647,-1.008196,-1.059415,-0.179687,-0.272058,5.788956,-1.295928,0,0,0,0,0


# SVC

In [138]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
                                      
dic={}
for i in range(len(param_range)):
    svm=SVC(kernel='linear', C=param_range[i], random_state=1)
    svm.fit(X_train_std, y_train)
    score1 = svm.score(X_test_std, y_test)
    score2 = svm.score(X_train_std, y_train)
    dic[svm]=score1

    if score1>max_score:
        val=cross_val_score(estimator=svm, X=X_train_std, y=y_train, cv=3)
        val1=np.mean(val)
        print('params: %s, train: %3f, test: %3f, val: %3f' %(svm, score2, score1, val1))

params: SVC(C=0.0001, kernel='linear', random_state=1), train: 0.552632, test: 0.545455, val: 0.552680
params: SVC(C=0.001, kernel='linear', random_state=1), train: 0.552632, test: 0.545455, val: 0.552680
params: SVC(C=0.01, kernel='linear', random_state=1), train: 0.598684, test: 0.560606, val: 0.546144
params: SVC(C=0.1, kernel='linear', random_state=1), train: 0.651316, test: 0.606061, val: 0.572810
params: SVC(kernel='linear', random_state=1), train: 0.677632, test: 0.666667, val: 0.546667
params: SVC(C=10.0, kernel='linear', random_state=1), train: 0.690789, test: 0.681818, val: 0.526667
params: SVC(C=100.0, kernel='linear', random_state=1), train: 0.717105, test: 0.696970, val: 0.533595
params: SVC(C=1000.0, kernel='linear', random_state=1), train: 0.710526, test: 0.712121, val: 0.520261


# ロジスティック回帰

In [83]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
                                      
dic={}
for i in range(len(param_range)):
    lr=LogisticRegression(C=param_range[i], random_state=1)
    lr.fit(X_train_std, y_train)
    score1 = lr.score(X_test_std, y_test)
    score2 = lr.score(X_train_std, y_train)
    dic[lr]=score1

    if score1>max_score:
        val=cross_val_score(estimator=lr, X=X_train_std, y=y_train, cv=3)
        val1=np.mean(val)
        print('params: %s, train: %3f, test: %3f, val: %3f' %(lr, score2, score1, val1))

params: LogisticRegression(C=0.0001, random_state=1), train: 0.552632, test: 0.545455, val: 0.552680
params: LogisticRegression(C=0.001, random_state=1), train: 0.552632, test: 0.530303, val: 0.552680
params: LogisticRegression(C=0.01, random_state=1), train: 0.611842, test: 0.560606, val: 0.572418
params: LogisticRegression(C=0.1, random_state=1), train: 0.638158, test: 0.575758, val: 0.592418
params: LogisticRegression(random_state=1), train: 0.690789, test: 0.560606, val: 0.572680


C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

params: LogisticRegression(C=10.0, random_state=1), train: 0.690789, test: 0.636364, val: 0.500915
params: LogisticRegression(C=100.0, random_state=1), train: 0.703947, test: 0.696970, val: 0.527190
params: LogisticRegression(C=1000.0, random_state=1), train: 0.710526, test: 0.681818, val: 0.533856


C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

# random forest

In [156]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.7
SearchMethod = 0
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
           "criterion": ["gini", "entropy"],
           "max_depth":[i for i in range(1, 21)]
          }
           
                                      
dic={}
for i in range(len(RFC_grid['n_estimators'])):
           for l in range(len(RFC_grid["criterion"])):
               for k in range(len(RFC_grid["max_depth"])):
                    forest=RandomForestClassifier(max_depth=RFC_grid["max_depth"][k], criterion=RFC_grid["criterion"][l], n_estimators=RFC_grid['n_estimators'][i], random_state=1)
                    forest.fit(X_train_std, y_train)
                    score=forest.score(X_train_std, y_train)
                    score1 = forest.score(X_test_std, y_test)
                    dic[forest]=score1

                    if score1>max_score:
                        val=cross_val_score(estimator=forest,
                                            X=X_train_std, y=y_train, cv=3)
                        val1=np.mean(val)
                        print('params: %s, train: %3f, test: %3f, val: %3f' %(forest, score, score1, val1))



params: RandomForestClassifier(max_depth=4, n_estimators=4, random_state=1), train: 0.809211, test: 0.712121, val: 0.533203
params: RandomForestClassifier(max_depth=8, n_estimators=7, random_state=1), train: 0.953947, test: 0.712121, val: 0.552941
params: RandomForestClassifier(max_depth=12, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552288
params: RandomForestClassifier(max_depth=13, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
params: RandomForestClassifier(max_depth=14, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
params: RandomForestClassifier(max_depth=15, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
params: RandomForestClassifier(max_depth=16, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
params: RandomForestClassifier(max_depth=17, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
pa

params: RandomForestClassifier(criterion='entropy', max_depth=18, n_estimators=15,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=15,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=15,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.598693
params: RandomForestClassifier(max_depth=4, n_estimators=16, random_state=1), train: 0.861842, test: 0.727273, val: 0.553464
params: RandomForestClassifier(max_depth=11, n_estimators=16, random_state=1), train: 0.986842, test: 0.712121, val: 0.585621
params: RandomForestClassifier(max_depth=12, n_estimators=16, random_state=1), train: 0.986842, test: 0.712121, val: 0.572418
params: RandomForestClassifier(max_depth=13, n_estimators=16, random_state=1), train: 0.993421, test: 0.727273, v

params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=18,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.579085
params: RandomForestClassifier(max_depth=4, n_estimators=19, random_state=1), train: 0.861842, test: 0.727273, val: 0.586405
params: RandomForestClassifier(max_depth=8, n_estimators=19, random_state=1), train: 0.993421, test: 0.712121, val: 0.539608
params: RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=19,
                       random_state=1), train: 0.967105, test: 0.742424, val: 0.572288
params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=19,
                       random_state=1), train: 0.973684, test: 0.787879, val: 0.546144
params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=19,
                       random_state=1), train: 0.993421, test: 0.757576, val: 0.519869
params: RandomForestClassifier(criterion='entropy', max_depth=10, n_estim

params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=21,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=21,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.592157
params: RandomForestClassifier(max_depth=4, n_estimators=22, random_state=1), train: 0.888158, test: 0.727273, val: 0.566275
params: RandomForestClassifier(max_depth=8, n_estimators=22, random_state=1), train: 0.993421, test: 0.727273, val: 0.565882
params: RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=22,
                       random_state=1), train: 0.973684, test: 0.712121, val: 0.559085
params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=22,
                       random_state=1), train: 0.973684, test: 0.787879, val: 0.552941
params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estim

params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=25,
                       random_state=1), train: 0.993421, test: 0.727273, val: 0.553072
params: RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=25,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.579216
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=25,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=25,
                       random_state=1), train: 1.000000, test: 0.757576, val: 0.585621
params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=25,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.605359
params: RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=25,
                       random_state=1), train: 1.000000, test: 0.757

...

params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=32,
                       random_state=1), train: 0.973684, test: 0.772727, val: 0.572418
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.727273, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.757576, val: 0.585490
params: RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.757576, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=16, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.757

params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=36,
                       random_state=1), train: 0.973684, test: 0.757576, val: 0.579216
params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=36,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.592288
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=36,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.605359
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=36,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.611765
params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=36,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.611895
params: RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=36,
                       random_state=1), train: 1.000000, test: 0.7424

params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=40,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.605490
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=40,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.605490
params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=41,
                       random_state=1), train: 0.973684, test: 0.772727, val: 0.592288
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=41,
                       random_state=1), train: 1.000000, test: 0.757576, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=41,
                       random_state=1), train: 1.000000, test: 0.727273, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=41,
                       random_state=1), train: 1.000000, test: 0.757

params: RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=45,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.579085
params: RandomForestClassifier(criterion='entropy', max_depth=16, n_estimators=45,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.579085
params: RandomForestClassifier(criterion='entropy', max_depth=17, n_estimators=45,
                       random_state=1), train: 1.000000, test: 0.712121, val: 0.579085
params: RandomForestClassifier(criterion='entropy', max_depth=18, n_estimators=45,
                       random_state=1), train: 1.000000, test: 0.712121, val: 0.579085
params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=45,
                       random_state=1), train: 1.000000, test: 0.712121, val: 0.579085
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=45,
                       random_state=1), train: 1.000000, test: 0.71

params: RandomForestClassifier(max_depth=13, n_estimators=50, random_state=1), train: 1.000000, test: 0.727273, val: 0.559477
params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=50,
                       random_state=1), train: 0.980263, test: 0.727273, val: 0.572680
params: RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=50,
                       random_state=1), train: 0.993421, test: 0.727273, val: 0.592418
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=50,
                       random_state=1), train: 1.000000, test: 0.727273, val: 0.572549
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=50,
                       random_state=1), train: 1.000000, test: 0.727273, val: 0.572680
params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=50,
                       random_state=1), train: 1.000000, test: 0.757576, val: 0.566144
params: RandomForestClass

In [74]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.70
SearchMethod = 0
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
           "criterion": ["gini", "entropy"],
           "max_depth":[i for i in range(1, 21)]
          }
           
                                      
dic={}
for i in range(len(RFC_grid['n_estimators'])):
           for l in range(len(RFC_grid["criterion"])):
               for k in range(len(RFC_grid["max_depth"])):
                    forest=RandomForestClassifier(max_depth=RFC_grid["max_depth"][k], criterion=RFC_grid["criterion"][l], n_estimators=RFC_grid['n_estimators'][i], random_state=1)
                    forest.fit(X_train_std, y_train)
                    score=forest.score(X_train_std, y_train)
                    score1 = forest.score(X_test_std, y_test)
                    dic[forest]=score1

                    if score1>max_score:
                        val=cross_val_score(estimator=forest,
                                            X=X_train_std, y=y_train, cv=3)
                        val1=np.mean(val)
                        print('params: %s, train: %3f, test: %3f, val: %3f' %(forest, score, score1, val1))


params: RandomForestClassifier(max_depth=4, n_estimators=4, random_state=1), train: 0.809211, test: 0.712121, val: 0.533203
params: RandomForestClassifier(max_depth=8, n_estimators=7, random_state=1), train: 0.953947, test: 0.712121, val: 0.552941
params: RandomForestClassifier(max_depth=12, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552288
params: RandomForestClassifier(max_depth=13, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
params: RandomForestClassifier(max_depth=14, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
params: RandomForestClassifier(max_depth=15, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
params: RandomForestClassifier(max_depth=16, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
params: RandomForestClassifier(max_depth=17, n_estimators=7, random_state=1), train: 0.967105, test: 0.712121, val: 0.552549
pa

params: RandomForestClassifier(criterion='entropy', max_depth=18, n_estimators=15,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=15,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=15,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.598693
params: RandomForestClassifier(max_depth=4, n_estimators=16, random_state=1), train: 0.861842, test: 0.727273, val: 0.553464
params: RandomForestClassifier(max_depth=11, n_estimators=16, random_state=1), train: 0.986842, test: 0.712121, val: 0.585621
params: RandomForestClassifier(max_depth=12, n_estimators=16, random_state=1), train: 0.986842, test: 0.712121, val: 0.572418
params: RandomForestClassifier(max_depth=13, n_estimators=16, random_state=1), train: 0.993421, test: 0.727273, v

params: RandomForestClassifier(max_depth=4, n_estimators=19, random_state=1), train: 0.861842, test: 0.727273, val: 0.586405
params: RandomForestClassifier(max_depth=8, n_estimators=19, random_state=1), train: 0.993421, test: 0.712121, val: 0.539608
params: RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=19,
                       random_state=1), train: 0.967105, test: 0.742424, val: 0.572288
params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=19,
                       random_state=1), train: 0.973684, test: 0.787879, val: 0.546144
params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=19,
                       random_state=1), train: 0.993421, test: 0.757576, val: 0.519869
params: RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=19,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.565882
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estim

params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=21,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=21,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.592157
params: RandomForestClassifier(max_depth=4, n_estimators=22, random_state=1), train: 0.888158, test: 0.727273, val: 0.566275
params: RandomForestClassifier(max_depth=8, n_estimators=22, random_state=1), train: 0.993421, test: 0.727273, val: 0.565882
params: RandomForestClassifier(criterion='entropy', max_depth=7, n_estimators=22,
                       random_state=1), train: 0.973684, test: 0.712121, val: 0.559085
params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=22,
                       random_state=1), train: 0.973684, test: 0.787879, val: 0.552941
params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estim

params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=25,
                       random_state=1), train: 0.993421, test: 0.727273, val: 0.553072
params: RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=25,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.579216
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=25,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=25,
                       random_state=1), train: 1.000000, test: 0.757576, val: 0.585621
params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=25,
                       random_state=1), train: 1.000000, test: 0.772727, val: 0.605359
params: RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=25,
                       random_state=1), train: 1.000000, test: 0.757

params: RandomForestClassifier(criterion='entropy', max_depth=19, n_estimators=28,
                       random_state=1), train: 0.993421, test: 0.742424, val: 0.585490
params: RandomForestClassifier(criterion='entropy', max_depth=20, n_estimators=28,
                       random_state=1), train: 0.993421, test: 0.742424, val: 0.585490
params: RandomForestClassifier(max_depth=7, n_estimators=29, random_state=1), train: 0.973684, test: 0.727273, val: 0.566144
params: RandomForestClassifier(max_depth=13, n_estimators=29, random_state=1), train: 0.993421, test: 0.757576, val: 0.578954
params: RandomForestClassifier(max_depth=15, n_estimators=29, random_state=1), train: 0.993421, test: 0.712121, val: 0.585490
params: RandomForestClassifier(max_depth=16, n_estimators=29, random_state=1), train: 0.993421, test: 0.712121, val: 0.585490
params: RandomForestClassifier(max_depth=17, n_estimators=29, random_state=1), train: 0.993421, test: 0.712121, val: 0.585490
params: RandomForestClassifier(

params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=32,
                       random_state=1), train: 0.973684, test: 0.772727, val: 0.572418
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.727273, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.598693
params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.757576, val: 0.585490
params: RandomForestClassifier(criterion='entropy', max_depth=15, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.757576, val: 0.592157
params: RandomForestClassifier(criterion='entropy', max_depth=16, n_estimators=32,
                       random_state=1), train: 1.000000, test: 0.757

params: RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=36,
                       random_state=1), train: 0.973684, test: 0.757576, val: 0.579216
params: RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=36,
                       random_state=1), train: 0.993421, test: 0.712121, val: 0.592288
params: RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=36,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.605359
params: RandomForestClassifier(criterion='entropy', max_depth=13, n_estimators=36,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.611765
params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=36,
                       random_state=1), train: 1.000000, test: 0.742424, val: 0.611895


KeyboardInterrupt: 

# Boosted forest

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
RFC_grid ={"base_estimator" : [DecisionTreeClassifier(max_depth=x) 
                                  for x in range(5, 10)],
              "learning_rate" : [0.5, 1.0, 1.5],
              "n_estimators": [i for i in range(1, 100)],
           }   
                                      
dic={}
for i in range(len(RFC_grid['base_estimator'])):
           for l in range(len(RFC_grid["learning_rate"])):
               for k in range(len(RFC_grid["n_estimators"])):
                    ada = AdaBoostClassifier(base_estimator=RFC_grid["base_estimator"][i], learning_rate=RFC_grid["learning_rate"][l], n_estimators=RFC_grid['n_estimators'][k], random_state=1)
                    ada.fit(X_train_std, y_train)
                    score1 = ada.score(X_test_std, y_test)
                    score2 = ada.score(X_train_std, y_train)
                    dic[ada]=score1

                    print('params: %s, train: %3f, test: %3f, val: %3f' %(ada, score2, score1, val1))



In [17]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.65
SearchMethod = 0
RFC_grid ={"base_estimator" : [DecisionTreeClassifier(max_depth=x) 
                                  for x in range(5, 10)],
              "learning_rate" : [0.5, 1.0, 1.5],
              "n_estimators": [i for i in range(1, 100)],
           }   
                                      
dic={}
for i in range(len(RFC_grid['base_estimator'])):
           for l in range(len(RFC_grid["learning_rate"])):
               for k in range(len(RFC_grid["n_estimators"])):
                    ada = AdaBoostClassifier(base_estimator=RFC_grid["base_estimator"][i], learning_rate=RFC_grid["learning_rate"][l], n_estimators=RFC_grid['n_estimators'][k], random_state=1)
                    ada.fit(X_train_std, y_train)
                    score1 = ada.score(X_test_std, y_test)
                    dic[ada]=score1

                    if score1>max_score:
                        val=cross_val_score(estimator=ada,
                                            X=X_train_std, y=y_train,
                                            cv=3)
                        val1=np.mean(val)
                        print('params: %s, score: %3f, val: %3f' %(ada, score1, val1))


params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=99, random_state=1), score: 0.651515, val: 0.565882
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=1.5, n_estimators=99, random_state=1), score: 0.666667, val: 0.552418
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   n_estimators=99, random_state=1), score: 0.727273, val: 0.558824
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7),
                   learning_rate=1.5, n_estimators=99, random_state=1), score: 0.666667, val: 0.546797
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=8),
                   learning_rate=1.5, n_estimators=99, random_state=1), score: 0.651515, val: 0.467190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimator

In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
RFC_grid ={"base_estimator" : [DecisionTreeClassifier(max_depth=x) 
                                  for x in range(5, 10)],
              "learning_rate" : [0.5, 1.0, 1.5],
              "n_estimators": [i for i in range(1, 100)],
           }   
                                      
dic={}
for i in range(len(RFC_grid['base_estimator'])):
           for l in range(len(RFC_grid["learning_rate"])):
               for k in range(len(RFC_grid["n_estimators"])):
                    ada = AdaBoostClassifier(base_estimator=RFC_grid["base_estimator"][i], learning_rate=RFC_grid["learning_rate"][l], n_estimators=RFC_grid['n_estimators'][k], random_state=1)
                    ada.fit(X_train_std, y_train)
                    score1 = ada.score(X_test_std, y_test)
                    dic[ada]=score1
                    
                    val=cross_val_score(estimator=ada,
                                        X=X_train_std, y=y_train,
                                        cv=3)
                    val1=np.mean(val)
                    print('params: %s, score: %3f, val: %3f' %(ada, score1, val1))

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=1, random_state=1), score: 0.636364, val: 0.533072
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=2, random_state=1), score: 0.621212, val: 0.526536
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=3, random_state=1), score: 0.666667, val: 0.526536
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=4, random_state=1), score: 0.696970, val: 0.539346
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=5, random_state=1), score: 0.606061, val: 0.499739
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=47, random_state=1), score: 0.681818, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=48, random_state=1), score: 0.681818, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=49, random_state=1), score: 0.681818, val: 0.506405
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, random_state=1), score: 0.681818, val: 0.506275
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=51, random_state=1), score: 0.681818, val: 0.506275
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=93, random_state=1), score: 0.651515, val: 0.506144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=94, random_state=1), score: 0.651515, val: 0.506144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=95, random_state=1), score: 0.651515, val: 0.506144
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=96, random_state=1), score: 0.651515, val: 0.499477
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.5, n_estimators=97, random_state=1), score: 0.636364, val: 0.512680
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
           

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=44, random_state=1), score: 0.666667, val: 0.592026
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=45, random_state=1), score: 0.666667, val: 0.592026
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=46, random_state=1), score: 0.651515, val: 0.572418
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=47, random_state=1), score: 0.651515, val: 0.572418
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=48, random_state=1), score: 0.666667, val: 0.565882
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=49, random_state=1), score: 0.681818, val: 0.565882
params: AdaBoostClassi

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=95, random_state=1), score: 0.666667, val: 0.565752
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=96, random_state=1), score: 0.651515, val: 0.572418
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=97, random_state=1), score: 0.666667, val: 0.559216
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=98, random_state=1), score: 0.651515, val: 0.585621
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=99, random_state=1), score: 0.651515, val: 0.565882
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=1.5, n_estimators=1, random_state=1), score: 0.636364, val: 0.533072
para

...

                   learning_rate=0.5, n_estimators=54, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=55, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=56, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=57, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=58, random_state=1), score: 0.530303, val: 0.500000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=0.5, n_estimators=59, random_state=1), score: 0.530303, v

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=3, random_state=1), score: 0.621212, val: 0.519739
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=4, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=5, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=6, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=7, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=8, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(b

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=56, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=57, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=58, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=59, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=60, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   n_estimators=61, random_state=1), score: 0.590909, val: 0.512941
params: AdaBoostClassi

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=9, random_state=1), score: 0.621212, val: 0.440654
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=10, random_state=1), score: 0.651515, val: 0.447320
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=11, random_state=1), score: 0.621212, val: 0.453856
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=12, random_state=1), score: 0.666667, val: 0.447190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=13, random_state=1), score: 0.666667, val: 0.467190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
            

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=55, random_state=1), score: 0.651515, val: 0.487190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=56, random_state=1), score: 0.651515, val: 0.500523
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=57, random_state=1), score: 0.651515, val: 0.507190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=58, random_state=1), score: 0.621212, val: 0.507190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
                   learning_rate=1.5, n_estimators=59, random_state=1), score: 0.651515, val: 0.507190
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=9),
           